In [1]:
import cv2
import mediapipe as mp
import os

In [3]:
def youtube_control(action):
    if action == "play_pause":
        os.system("osascript -e 'tell application \"System Events\" to keystroke \" \"'")  
    elif action == "volume_up":
        os.system("osascript -e 'tell application \"System Events\" to key code 126'")  
    elif action == "volume_down":
        os.system("osascript -e 'tell application \"System Events\" to key code 125'") 
    elif action == "next_video":
        os.system("osascript -e 'tell application \"System Events\" to keystroke \"n\" using shift down'") 
    elif action == "prev_video":
        os.system("osascript -e 'tell application \"System Events\" to keystroke \"p\" using shift down'")  
    elif action == "stop":
        os.system("osascript -e 'tell application \"System Events\" to keystroke \"s\"'") 

In [4]:
def spotify_control(action):
    if action == "play_pause":
        os.system("osascript -e 'tell application \"Spotify\" to playpause'") 
    elif action == "volume_up":
        os.system("osascript -e 'tell application \"Spotify\" to set sound volume to (sound volume + 10)'") 
    elif action == "volume_down":
        os.system("osascript -e 'tell application \"Spotify\" to set sound volume to (sound volume - 10)'")  
    elif action == "next_track":
        os.system("osascript -e 'tell application \"Spotify\" to next track'")  
    elif action == "prev_track":
        os.system("osascript -e 'tell application \"Spotify\" to previous track'")

In [5]:
def handle_media_control(gesture, platform):
    if platform == "YouTube":
        youtube_control(gesture)
    elif platform == "Spotify":
        spotify_control(gesture)

In [6]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

I0000 00:00:1742360632.825046   97062 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742360632.841664   97816 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742360632.848819   97816 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  

    results = hands.process(rgb_frame)  

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            thumb_tip = hand_landmarks.landmark[4]
            index_tip = hand_landmarks.landmark[8]
            middle_tip = hand_landmarks.landmark[12]
            ring_tip = hand_landmarks.landmark[16]
            pinky_tip = hand_landmarks.landmark[20]

            if (thumb_tip.y < index_tip.y and
                thumb_tip.y < middle_tip.y and
                thumb_tip.y < ring_tip.y and
                thumb_tip.y < pinky_tip.y):
                print("Play/Pause Gesture Detected")
                handle_media_control("play_pause", "Spotify")

            elif (index_tip.y < middle_tip.y and
                  ring_tip.y > middle_tip.y and
                  pinky_tip.y > middle_tip.y):
                print("Next Track Gesture Detected")
                handle_media_control("next_track", "Spotify")

            elif (thumb_tip.y < index_tip.y and
                  thumb_tip.x < index_tip.x and
                  index_tip.y < middle_tip.y and
                  ring_tip.y < pinky_tip.y):
                print("Volume Up Gesture Detected")
                handle_media_control("volume_up", "Spotify")

            elif (thumb_tip.y > index_tip.y and
                  thumb_tip.x < index_tip.x and
                  index_tip.y < middle_tip.y and
                  ring_tip.y < pinky_tip.y):
                print("Volume Down Gesture Detected")
                handle_media_control("volume_down", "Spotify")

    cv2.imshow("mova", frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
